# Algoritmo de Grover

- Referência: https://www.youtube.com/watch?v=0RPFWZj7Jm0
- Sobre os simuladores: https://qiskit.github.io/qiskit-aer/tutorials/1_aersimulator.html

In [23]:
my_list = [1, 3, 5, 2, 4, 9, 5, 8, 0, 7, 6]

print(my_list)

def the_oracle(my_input):
    winner = 7
    return my_input is winner

for index, trial_number in enumerate(my_list):
    if the_oracle(trial_number) is True:
        print(f"Winner found at index {index}")
        print(f"{index+1} calls to Oracle used")
        break

[1, 3, 5, 2, 4, 9, 5, 8, 0, 7, 6]
Winner found at index 9
10 calls to Oracle used


In [24]:
from qiskit import *
from qiskit_aer import AerSimulator
import numpy as np

# define the oracle circuit

oracle = QuantumCircuit(2, name='Oracle')
oracle.cz(0,1)
oracle.to_gate()

oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

In [25]:
grover_circ = QuantumCircuit(2, 2)

grover_circ.h([0,1])
grover_circ.append(oracle, [0,1])

grover_circ.draw()

┌───┐┌─────────┐
q_0: ┤ H ├┤0        ├
     ├───┤│  Oracle │
q_1: ┤ H ├┤1        ├
     └───┘└─────────┘
c: 2/════════════════

In [26]:
grover_circ.save_statevector()

sim_statevector = AerSimulator(method='statevector')
grover_circ = transpile(grover_circ, sim_statevector)

job_statevector = sim_statevector.run(grover_circ, shots=10000)
result = job_statevector.result()
sv = result.get_statevector(0)
np.round(sv, 2)


array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

In [27]:
reflection = QuantumCircuit(2, name='reflection')

reflection.h([0,1])
reflection.z([0,1])
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()

reflection.draw()

┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [28]:
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle, [0,1])
grover_circ.append(reflection, [0,1])
grover_circ.measure([0,1], [0,1])

grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  Oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [29]:
simulator = AerSimulator()
grover_circ = transpile(grover_circ, simulator)

job = simulator.run(grover_circ, shots=1)
result = job.result()
result.get_counts()

{'11': 1}